In [162]:
from pythreejs import *
from IPython.display import display
import sys, os, random, time
from math import *
import ode

# Collision callback for pyode
def near_callback(args, geom1, geom2):
    """Callback function for the collide() method.

    This function checks if the given geoms do collide and
    creates contact joints if they do.
    """

    # Check if the objects do collide
    contacts = ode.collide(geom1, geom2)

    # Create contact joints
    world,contactgroup = args
    for c in contacts:
        c.setBounce(0.2)
        c.setMu(5000)
        j = ode.ContactJoint(world, contactgroup, c)
        j.attach(geom1.getBody(), geom2.getBody())


x = 0
y = 0
width = 640
height = 480
g = (0,0,-9.8)
x_ll = (0.0,0.0,0.0)
L = (1.0,1.0,1.0)

world = ode.World()
world.setERP(0.5)
world.setCFM(1E-5)
world.setGravity(g)

space = ode.Space()

eps_x = L[0]- 0.99*L[0]
eps_y = L[1]- 0.99*L[1]
tankWalls = [ode.GeomPlane(space, (0,0,1) ,x_ll[2]),
ode.GeomPlane(space, (1,0,0) ,x_ll[0]+eps_x),
ode.GeomPlane(space, (-1,0,0),-(x_ll[0]+L[0]-eps_x)),
ode.GeomPlane(space, (0,1,0) ,x_ll[1]+eps_y),
ode.GeomPlane(space, (0,-1,0) ,-(x_ll[1]+L[1]-eps_y))]

contactgroup = ode.JointGroup()

bar_dim = (0.33,0.33,0.33)
bar_center = (0.5,0.5,0.6)
density=500.0
M = ode.Mass()
M.setBox(density,bar_dim[0],bar_dim[1],bar_dim[2])

body = ode.Body(world)
body.setMass(M)
body.shape="box"
body.boxsize=bar_dim

bar = ode.GeomBox(space,bar_dim)
bar.setBody(body)
bar.setPosition(bar_center) 

bodies = [body]

geoms = [bar]


# Some variables used inside the simulation loop
fps = 20
dt = 1.0/fps
running = True
state = 0
counter = 0
objcount = 0
lasttime = time.time()


for b in bodies:
    sx,sy,sz = b.boxsize
    b.widget = Mesh(geometry=BoxGeometry(width=sx,height=sy,depth=sz),
                    material=LambertMaterial(color='aqua'),
                    position=b.getPosition())
    b.widget.quaternion_from_rotation(b.getRotation())

children=[b.widget for  b  in bodies]
children.append(Mesh(geometry=BoxGeometry(width=1.,height=1.,depth=1.), 
                     material=LambertMaterial(color=0xffffff, #color=0xccccff, 
                                              opacity=0.7,
                                              refractionRatio=0.985, 
                                              reflectivity= 0.9,
                                              transparent=True),
                     position=(.5,.5,.5)))
#children.append(Mesh(geometry=PlaneGeometry(width=10,height=10),material=BasicMaterial(color='blue')))
children.append(AmbientLight(color=0x777777))
scene = Scene(children=children)
c = PerspectiveCamera(target=(0.5,0.5,0.5),position=[3,0,0], up=[0,0,1], children=[DirectionalLight(color='white', 
                                                                             position=[3,1,5], 
                                                                             intensity=0.5)])
renderer = Renderer(camera=c, scene = scene, controls=[OrbitControls(controlling=c)])
renderer.background='gray'
ar = float(renderer.height)/(renderer.width)
renderer.width = 600
renderer.height = int(ar*renderer.width)
display(renderer)
lasttime = time.time()
for i in range(300):
    t = dt - (time.time() - lasttime)
    if (i==30):
        world.setGravity((0.0,9.8,0.0))
    if (i==60):
        world.setGravity((0.0,-9.8,0.0))
    if (i==90):
        world.setGravity((9.8,0.0,0.0))
    if (i==120):
        world.setGravity((-9.8,0.0,0.8))
    if (i==150):
        world.setGravity((0.0,0.0,9.8))
    if (i>=165):
        world.setGravity((0.0,0.0,-9.8))
        for b in bodies:
            x,y,z = b.getPosition()
            waterLevel = 0.5
            if waterLevel - z > 0:
                b.setForce((0.0,0.0,(waterLevel-z)*bar_dim[0]*bar_dim[1]*1000.0*9.8))
    if (t > 0):
        time.sleep(t)
        space.collide((world,contactgroup), near_callback)
        # Simulation step
        world.step(dt)
        # Remove all contact joints
        contactgroup.empty()
        for b in bodies:
            x,y,z = b.getPosition()
            R = b.getRotation()
            if b.shape=="box":
                b.widget.position=(x,y,z)
                b.widget.quaternion_from_rotation(R)
    lasttime = time.time()